In [ ]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Flatten
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils import np_utils
from keras.models import Sequential
from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers.convolutional import Convolution2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os
from keras.layers import Activation, Conv2D, Dropout
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D,AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling1D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_rows=224
num_classes=8  
img_cols=224
num_channel=3

import splitfolders
splitfolders.ratio(
    input='/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/gz2_images_dataset07',
    output="/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/Galaxy-Classification-Using-CNN/output_dataset",
    seed=1337,
    ratio=(0.79,0.01,0.2)  
)

TRAINING_DIR = "/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/Galaxy-Classification-Using-CNN/output_dataset/train"

train_datagen = ImageDataGenerator(rescale = 1/255.0)  

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size=8,  
    class_mode='categorical',
    target_size=(img_rows, img_cols),
    shuffle=True
)

validation_DIR = '/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/Galaxy-Classification-Using-CNN/output_dataset/val'

validation_datagen = ImageDataGenerator(rescale = 1/255.0)

validation_generator = validation_datagen.flow_from_directory(
    validation_DIR,
    batch_size=8,  
    class_mode='categorical',
    target_size=(img_rows, img_cols)
)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1/255.0)  

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    batch_size=8,  
    class_mode='categorical',
    target_size=(img_rows, img_cols),
    shuffle=True
)


## ECA module Based CNN Model

In [ ]:
import tensorflow as tf
from tensorflow import keras

input_shape=(224,224,3)
##mini_exceptin

img_input = Input(input_shape)
x = SeparableConv2D(32, (3, 3), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",kernel_regularizer='l1', use_bias=False)(img_input)
x = BatchNormalization()(x)
x_offset = Activation('relu')(x)


x_offset


In [ ]:
# module 1
residual = SeparableConv2D(32, (3, 3), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",kernel_regularizer='l1', use_bias=False)(x_offset)
residual = BatchNormalization()(residual)
residual=Activation('relu')(residual)



residual.shape

In [ ]:
residual1 = SeparableConv2D(32, (5, 5), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",kernel_regularizer='l1', use_bias=False)(x_offset)
residual1 = BatchNormalization()(residual1)
residual1=Activation('relu')(residual1)



residual1.shape

In [ ]:
# module 1
residual2 = SeparableConv2D(32, (7, 7), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",kernel_regularizer='l1', use_bias=False)(x_offset)
residual2 = BatchNormalization()(residual2)
residual2=Activation('relu')(residual2)

In [ ]:
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling2D, Reshape, Multiply

In [ ]:
def eca_module(x, k_size=5):
    squeeze = GlobalAveragePooling2D()(x)
    attn = Conv1D(filters=1,
                            kernel_size=k_size,
                            padding='same',
                            kernel_initializer='he_normal',
                            use_bias=False)(tf.expand_dims(squeeze, axis=1))
    attn = tf.math.sigmoid(tf.expand_dims(tf.transpose(attn, [0, 2, 1]), axis=3))
    scale = x * attn
    return scale

In [ ]:

channel_attention_map=eca_module(x_offset)
channel_attention_map

In [ ]:
x11 = layers.add([residual1,residual,residual2,channel_attention_map])
x11.shape

## Dilation Convolution Layer

In [ ]:
x =  SeparableConv2D(64, (3, 3), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",kernel_regularizer='l1', use_bias=False)(x11)
x = BatchNormalization()(x)
x_offset = Activation('relu')(x)
x_offset = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x_offset)

x_offset

In [ ]:
# module 1
residual = SeparableConv2D(64, (3, 3), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",activation='relu',kernel_regularizer='l1', use_bias=False)(x_offset)
residual = BatchNormalization()(residual)
residual=Activation('relu')(residual)



residual.shape

In [ ]:
residual1 = SeparableConv2D(64, (5, 5), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",activation='relu',kernel_regularizer='l1', use_bias=False)(x_offset)
residual1 = BatchNormalization()(residual1)
residual2=Activation('relu')(residual1)



residual2.shape

In [ ]:
# module 1
residual3 = SeparableConv2D(64, (7, 7), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",activation='relu',kernel_regularizer='l1', use_bias=False)(x_offset)
residual3 = BatchNormalization()(residual3)
residual3=Activation('relu')(residual3)

In [ ]:
def eca_module(x, k_size=5):
    squeeze = GlobalAveragePooling2D()(x)
    attn = Conv1D(filters=1,
                            kernel_size=k_size,
                            padding='same',
                            kernel_initializer='he_normal',
                            use_bias=False)(tf.expand_dims(squeeze, axis=1))
    attn = tf.math.sigmoid(tf.expand_dims(tf.transpose(attn, [0, 2, 1]), axis=3))
    scale = x * attn
    return scale

In [ ]:

channel_attention_map=eca_module(x_offset)
channel_attention_map

In [ ]:
x11 = layers.add([residual2,residual,channel_attention_map,residual3])
x11.shape

## Dilation Convolution Layer

In [ ]:
x =  SeparableConv2D(128, (3, 3), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",kernel_regularizer='l1', use_bias=False)(x11)
x = BatchNormalization()(x)
x_offset = Activation('relu')(x)
x_offset = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x_offset)

x_offset

In [ ]:
# module 1
residual = SeparableConv2D(128, (3, 3), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",kernel_regularizer='l1', use_bias=False)(x_offset)
residual = BatchNormalization()(residual)
residual=Activation('relu')(residual)



residual.shape

In [ ]:
# module 1
residual1 = SeparableConv2D(128, (5, 5), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",kernel_regularizer='l1', use_bias=False)(x_offset)
residual1 = BatchNormalization()(residual1)
residual12=Activation('relu')(residual1)

In [ ]:
residual1 = SeparableConv2D(128, (5, 5), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",kernel_regularizer='l1', use_bias=False)(x_offset)
residual1 = BatchNormalization()(residual1)
residual13=Activation('relu')(residual1)

In [ ]:
def eca_module(x, k_size=5):
    squeeze = GlobalAveragePooling2D()(x)
    attn = Conv1D(filters=1,
                            kernel_size=k_size,
                            padding='same',
                            kernel_initializer='he_normal',
                            use_bias=False)(tf.expand_dims(squeeze, axis=1))
    attn = tf.math.sigmoid(tf.expand_dims(tf.transpose(attn, [0, 2, 1]), axis=3))
    scale = x * attn
    return scale

In [ ]:

channel_attention_map=eca_module(x_offset)
channel_attention_map

In [ ]:

x11 = layers.add([residual12,residual,residual13,channel_attention_map])
x11.shape

In [ ]:
x =  SeparableConv2D(256, (3, 3), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",kernel_regularizer='l1', use_bias=False)(x11)
x = BatchNormalization()(x)
x_offset = Activation('relu')(x)

x_offset

## Dilation Convolution Layer

In [ ]:
x =  SeparableConv2D(256, (3, 3), strides=(1, 1), padding='same',depthwise_initializer='he_normal',pointwise_initializer="he_normal",kernel_regularizer='l1', use_bias=False)(x11)
x = BatchNormalization()(x)
x_offset = Activation('relu')(x)

x_offset

In [ ]:
x15=GlobalAveragePooling2D()(x_offset)
x15

In [ ]:
x = tf.keras.layers.Dropout(0.3)(x15)

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Dense, concatenate



In [ ]:
output=Dense(8, activation='softmax')(x)
output

In [ ]:
model = Model(img_input, output)

model.summary()

# DIAT-DSCNN-ECA on GZ2

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

import tensorflow as tf
from tensorflow import keras

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),metrics=["accuracy"])

In [ ]:
def lr_schedule(epoch):
  """
  Returns a custom learning rate that decreases as epochs progress.
  """
  learning_rate = 0.01
  if epoch > 10:
    learning_rate = 0.002
  if epoch > 25:
    learning_rate = 0.001
  if epoch > 35:
    learning_rate = 0.0001

  tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
  return learning_rate

In [ ]:
from tensorflow import keras
from tensorflow import keras
lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)
model.get_config()
model.layers[0].get_config()
model.layers[0].input_shape
model.layers[0].output_shape
model.layers[0].get_weights()
#np.shape(model.layers[0].get_weights()[0])
model.layers[0].trainable


In [ ]:
#history = model.fit(train_generator,validation_data = validation_generator,steps_per_epoch=3482//8,validation_steps=745//8,epochs=100,batch_size=16, verbose=1)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
history = model.fit(train_generator,validation_data = validation_generator,epochs=80,batch_size=32, verbose=1,callbacks=[ lr_callback])


In [ ]:

model.save('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/Galaxy-Classification-Using-CNN/New_gz2-path-2block_adam_CNN_0.7_0.15_0.15_224x224xx3_30.h5')

In [ ]:
from tensorflow.keras.models import load_model
# new_model = load_model('/content/drive/MyDrive/galaxyzoo2_3block_adam_with_ECA_256x256x3_k=5_0.7_0.15_0.15.h5')
model = load_model('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/Galaxy-Classification-Using-CNN/New_gz2-path-2block_adam_CNN_0.7_0.15_0.15_224x224xx3_30.h5')


In [ ]:
import os
import numpy as np
from tqdm import tqdm  

img_height = 224
img_width = 224
main = '/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/Galaxy-Classification-Using-CNN/output_dataset/test'

sorted_classes = sorted(os.listdir(main))  

test_preprocessed_images = []

for class_name in tqdm(sorted_classes, desc='Processing Classes'):
    class_dir = os.path.join(main, class_name)
    if not os.path.isdir(class_dir):
        continue  
    
    image_files = sorted(os.listdir(class_dir))  
    for img_name in tqdm(image_files, desc=f'Class {class_name}', leave=False):
        img_path = os.path.join(class_dir, img_name)
        processed = preprocess_image(img_path)  
        test_preprocessed_images.append(processed)

test_preprocessed_images = np.vstack(test_preprocessed_images)
np.save('/remote-home/cs_acmis_hby/Galaxy-Zoo-Classification/Contrast_experiment/Galaxy-Classification-Using-CNN/test_preproc_CNN.npy', test_preprocessed_images)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_true, answer)
conf_mat

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Reds):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = cm.round(2)
        #print("Normalized confusion matrix")
    else:
        cm=cm
        #print('Confusion matrix, without normalization')

    #print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools  

conf_mat = confusion_matrix(y_true, answer)

class_names = [
    "barred_spirals",          # 0
    "cigar_shaped_elliptical", # 1
    "edge_on",                 # 2
    "in_between_elliptical",   # 3
    "irregular",               # 4
    "merger",                  # 5
    "round_elliptical",        # 6
    "unbarred_spirals"         # 7
]

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.round(cm, 2)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')

plt.figure(figsize=(14, 12)) 
plot_confusion_matrix(conf_mat, class_names, normalize=True, cmap=plt.cm.Blues)
plt.show()